Several step 
  - Find all the url that we need to collect data from
  - Collect and clean the data 
  - More cleaning & transform the data into a match up graph (basicly network graph)
  - linear optimisation to find the probability I should be playing the character

Let's First find the url of all figther data we need to extract

In [53]:
# get match up data from sf5 
from BeautifulSoup import BeautifulSoup
import urllib2
import re
import pandas as pd
import numpy as np
import csv

# global variables
MAIN_URL = 'http://www.eventhubs.com'

# read the main url 
html_page = urllib2.urlopen("http://www.eventhubs.com/tiers/sf5/")
soup = BeautifulSoup(html_page)

# extract the fighter url page
url_raw = []
for link in soup.findAll('a'):
    href = link.get('href')
    href = str(href)
    if '/tiers/sf5/character/' in href and 'vote' not in href:
        url_raw.append(href)
        
url_fighter_list = [MAIN_URL + string_in_list for string_in_list in url_raw]
url_fighter_list = set(url_fighter_list)

# verify the url and the number of fighters
print len(url_fighter_list)
url_fighter_list

21


{'http://www.eventhubs.com/tiers/sf5/character/alex',
 'http://www.eventhubs.com/tiers/sf5/character/balrog',
 'http://www.eventhubs.com/tiers/sf5/character/birdie',
 'http://www.eventhubs.com/tiers/sf5/character/cammy',
 'http://www.eventhubs.com/tiers/sf5/character/chun-li',
 'http://www.eventhubs.com/tiers/sf5/character/dhalsim',
 'http://www.eventhubs.com/tiers/sf5/character/fang',
 'http://www.eventhubs.com/tiers/sf5/character/guile',
 'http://www.eventhubs.com/tiers/sf5/character/ibuki',
 'http://www.eventhubs.com/tiers/sf5/character/juri',
 'http://www.eventhubs.com/tiers/sf5/character/karin',
 'http://www.eventhubs.com/tiers/sf5/character/ken',
 'http://www.eventhubs.com/tiers/sf5/character/laura',
 'http://www.eventhubs.com/tiers/sf5/character/m-bison',
 'http://www.eventhubs.com/tiers/sf5/character/nash',
 'http://www.eventhubs.com/tiers/sf5/character/necalli',
 'http://www.eventhubs.com/tiers/sf5/character/r-mika',
 'http://www.eventhubs.com/tiers/sf5/character/rashid',
 'ht

Let's collect the data

In [72]:
# for every fighter url read all the table data 
# transform in 3 columns data : fighter_name , fighter_against_name, match_avg_score.
nb_fighter = len(url_fighter_list)

for row,url_fighter in enumerate(url_fighter_list):
    
    html_fighter_page = urllib2.urlopen(url_fighter)
    soup_fighter = BeautifulSoup(html_fighter_page)

    # find fighter opponents name
    fighter_against_list_name = []

    for html_row in soup_fighter.findAll('td'):
        html_row_text = str(html_row) 
        if '/tiers/sf5' in html_row_text:
            start_character_number = html_row_text.index('/tiers/sf5') + 26 
            end_character_number = html_row_text.index('</a>') 
            fighter_against = html_row_text[start_character_number:end_character_number]
            fighter_against_list_name.append(fighter_against)

    # find match avg score for every opponents
    # ad a constraints 3 digits
    match_score_list = []

    for html_row in soup_fighter.findAll('td'):
        html_row_text = str(html_row) 
        match_score_compile = re.search(r"[\d][.][\d]+", html_row_text)

        if match_score_compile is not None:
            match_score_text = match_score_compile.group()
            match_score_text_clean = match_score_text[:3]
            match_score_test_int = int(match_score_text[0]) 

            if match_score_test_int > 0 and len(match_score_text) == 3:
                match_score_list.append(match_score_text_clean)

    # attach fighter_name to data
    fighter_name = url_fighter[url_fighter.index('character')  + 10:]
    fighter_name = np.array([fighter_name])
    repeat_length = len(fighter_against_list_name)
    fighter_name_list = np.repeat(fighter_name, repeat_length, axis=0)

    # prepare the data
    balancing_raw_data = pd.DataFrame([fighter_name_list,fighter_against_list_name, match_score_list])
    balancing_raw_data_transpose = balancing_raw_data.T[0:nb_fighter - 1]
    balancing_raw_data_transpose.columns = ['fighter_A', 'fighter_B','match_score']
    
    # combine all the data
    if row == 0:
        fighter_data = balancing_raw_data_transpose
    else:
        fighter_data = fighter_data.append(balancing_raw_data_transpose)    
        
# lower fighter name 
fighter_data['fighter_B'] = fighter_data['fighter_B'].str.lower()
fighter_data['fighter_A'] = fighter_data['fighter_A'].str.lower()


In [73]:
# verify if fighter name have the same name in both column
print(set(fighter_data['fighter_A']) == set(fighter_data['fighter_B']))

# fix name
fighter_data['fighter_B'] = fighter_data['fighter_B'].replace(to_replace='f.a.n.g', value='fang')
fighter_data['fighter_B'] = fighter_data['fighter_B'].replace(to_replace='m. bison', value='m-bison')
fighter_data['fighter_B'] = fighter_data['fighter_B'].replace(to_replace='r. mika', value='r-mika')

print(set(fighter_data['fighter_A']) == set(fighter_data['fighter_B']))

False
True


In [54]:
# verify the data at this step. sounds good
fighter_data

,fighter_A,fighter_B,match_score
0,dhalsim,zangief,7.0
1,dhalsim,birdie,5.9
2,dhalsim,laura,5.5
3,dhalsim,fang,5.2
4,dhalsim,guile,5.2
5,dhalsim,alex,5.2
6,dhalsim,m-bison,5.1
7,dhalsim,r-mika,5.0
8,dhalsim,nash,4.9
9,dhalsim,ryu,4.9


Now let's transform the data into a match-up chart

In [182]:
# get unique fighter name (using fighter_A name column)
# the matchup seems accurate
nrow = len(fighter_data)
nb_fighter = len(figther_name)
tableau = np.zeros([nb_fighter,nb_fighter])
figther_name = set(fighter_data['fighter_A'])

# fill the matchup chart
# the first two for loop generate all the possible combination of fighter_A and fighter_B
# the output set contain the 18 name of sf5 character 

for i,fighter_A in enumerate(fighter_name):
    for j,fighter_B in enumerate(fighter_name):
        
        # iterate throught the data x 
        # Find the combination of the fighter_A(i) and fighter_B(j) in the data x
        # Append the the score points to the i and j location in tableau[i,j]
        for row in range(0,nrow):
            if fighter_data.fighter_A.iloc[row] == fighter_A and \
               fighter_data.fighter_B.iloc[row] == fighter_B:
                    tableau[i,j] = fighter_data.match_score.iloc[row]
                    #print(fighter_data.fighter_B.iloc[row])
                    #print(fighter_data.fighter_A.iloc[row])
                    #print(fighter_data.match_score.iloc[row])
        
        # if figther_A and fighter_B are the same we assign the value 5.0 in our matrix
        if fighter_A == fighter_B:
            tableau[i,j] = 5.0

# retransform to pandas and ad column and row name
match_up_chart = pd.DataFrame(tableau)
match_up_chart = match_up_chart / 10

# rename row and columns
match_up_chart.columns = figther_name
match_up_chart.index = list(figther_name)

match_up_chart

,m-bison,ken,r-mika,alex,zangief,fang,rashid,cammy,chun-li,balrog,...,laura,birdie,juri,karin,dhalsim,vega,ryu,guile,nash,ibuki
m-bison,0.50,0.50,0.45,0.51,0.52,0.53,0.52,0.48,0.46,0.48,...,0.50,0.51,0.50,0.49,0.49,0.50,0.48,0.48,0.46,0.49
ken,0.50,0.50,0.51,0.55,0.58,0.54,0.52,0.51,0.45,0.51,...,0.55,0.52,0.51,0.49,0.52,0.52,0.47,0.52,0.46,0.56
r-mika,0.55,0.49,0.50,0.54,0.50,0.56,0.52,0.50,0.45,0.58,...,0.54,0.51,0.51,0.48,0.50,0.51,0.46,0.49,0.50,0.55
alex,0.49,0.45,0.46,0.50,0.52,0.51,0.50,0.47,0.39,0.47,...,0.49,0.50,0.45,0.47,0.48,0.48,0.43,0.45,0.41,0.51
zangief,0.48,0.42,0.50,0.48,0.50,0.40,0.40,0.48,0.32,0.52,...,0.51,0.41,0.46,0.43,0.30,0.41,0.39,0.32,0.35,0.52
fang,0.47,0.46,0.44,0.49,0.60,0.50,0.48,0.41,0.42,0.49,...,0.46,0.52,0.48,0.44,0.48,0.45,0.41,0.45,0.41,0.46
rashid,0.48,0.48,0.48,0.50,0.60,0.52,0.50,0.45,0.42,0.48,...,0.50,0.52,0.47,0.45,0.55,0.48,0.44,0.46,0.45,0.49
cammy,0.52,0.49,0.50,0.53,0.52,0.59,0.55,0.50,0.46,0.52,...,0.54,0.51,0.51,0.49,0.54,0.52,0.48,0.52,0.51,0.53
chun-li,0.54,0.55,0.55,0.61,0.68,0.58,0.58,0.54,0.50,0.57,...,0.61,0.59,0.56,0.52,0.52,0.55,0.51,0.54,0.52,0.61
balrog,0.52,0.49,0.42,0.53,0.48,0.51,0.52,0.48,0.43,0.50,...,0.47,0.49,0.50,0.45,0.53,0.48,0.46,0.47,0.43,0.56


In [184]:
# prepare the csv so that it works with function found on github 
# https://github.com/Blinkity/metagame/blob/master/bounds.py

match_up_chart.to_csv('sf_5_balancing.csv')
cols_position = [i for i in xrange(1,22,1)]
matchups = pd.read_csv('sf_5_balancing.csv',skiprows = 0,index_col = 0)
matchups.index.name = "row_char"
matchups.columns = matchups.index.values #need to use values so we can copy it and have two different names
matchups.columns.name = "col_char"

# data format in github
right_format = pd.read_csv('/Users/jpmallette/Downloads/metagame-master/matchups.csv',header = None,index_col = 0)
right_format

,1,2,3
0,,,
Seth,0.5,0.4,0.3
C. Viper,0.6,0.5,0.6
Cammy,0.7,0.4,0.5


In [ ]:
# We see there are some missing value, I will need to investigate this throught

Now let's do linear optimization to identify balancing opportunity.

In [187]:
import matplotlib
import pandas
import seaborn as sns
sns.set_context("talk")
import numpy as np
import pulp
from pulp import *


def makeMatchups(allRanks, selfRanks):
    overall_beat_probs = (3.0/2.0)*selfRanks['beat_opponent_prob'] - 0.25
    overall_beat_probs.name = "overall_beat_opponent_prob"
    overall_beat_probs_df = pandas.DataFrame(overall_beat_probs)
    mergedRanks = allRanks.merge(overall_beat_probs_df, left_on='PLAYER_CHAR_COPY', right_index=True)
    specific_opponent_probs = 3*mergedRanks['beat_opponent_prob'] - 2*mergedRanks['overall_beat_opponent_prob']
    matchups = specific_opponent_probs.unstack()
    averagedMatchups = (matchups + (1 - matchups.T))/2.0
    return averagedMatchups

def setupBasicProblem(matrix):
    prob = LpProblem("rock_paper_scissors", pulp.LpMaximize)
    the_vars = np.append(matrix.index.values, (["w"]))
    lp_vars = LpVariable.dicts("vrs", the_vars)
#First add the objective function.
    prob += lpSum([lp_vars['w']])  
#Now add the non-negativity constraints.
    for row_strat in matrix.index.values:
        prob += lpSum([1.0 * lp_vars[row_strat]]) >= 0
#Now add the sum=1 constraint.
    prob += lpSum([1.0 * lp_vars[x] for x in matrix.index.values]) == 1
#Now add the column payoff constraints
    for col_strat in matrix.columns.values:
        stratTerms = [matrix.loc[row_strat, col_strat] * lp_vars[row_strat] for row_strat in matrix.index.values]
        allTerms = stratTerms + [-1 * lp_vars['w']]
        prob += lpSum(allTerms) >= 0
#now write it out and solve
    return prob, lp_vars

def solveGame(matrix):
    prob, lp_vars = setupBasicProblem(matrix)
    prob.writeLP("rockpaperscissors.lp")
    prob.solve()
#now prepare the value and mixed strategy
    game_val = value(lp_vars['w'])
    strat_probs = {}
    for row_strat in matrix.index.values:
        strat_probs[row_strat] = value(lp_vars[row_strat])
#and output it
    return prob, game_val, strat_probs

def solveGameWithRowConstraint(matrix, rowname, constraint):
    prob, lp_vars = setupBasicProblem(matrix)
#add the additional constraint
    prob += lpSum(lp_vars[rowname]) == constraint
    prob.writeLP("rockpaperscissors.lp")
    prob.solve()
#now prepare the value and mixed strategy
    game_val = value(lp_vars['w'])
    strat_probs = {}
    for row_strat in matrix.index.values:
        strat_probs[row_strat] = value(lp_vars[row_strat])
#and output it
    return prob, game_val, strat_probs

def getWinRates(rowname,matrix,division=10):
    probs = np.linspace(0,1,division+1)
    return pandas.Series([solveGameWithRowConstraint(matrix, rowname, p)[1] for p in probs], index=probs, name=rowname)

def getAllWinRates(matrix,division=10):
    return pandas.concat([getWinRates(row,matrix,division) for row in matrix.index.values], axis=1)   

def plotIntervals(winRates,doSort,threshold):
    intervals = winRates.apply(lambda x: pandas.Series([x[x >= threshold].first_valid_index(), x[x >= threshold].last_valid_index()], index = ['minv','maxv'])).T
    intervals['bar1'] = intervals['minv']
    intervals['bar2'] = intervals['maxv'] - intervals['minv']
    intervals['bar3'] = 1 - (intervals['bar1'] + intervals['bar2'])
    #Maybe we want to sort by max, min values, or maybe we just want to keep it in its matchup-chart-specified order.
    if doSort:
        intervals = intervals.sort_index(by=['maxv','minv'])
    else: #else reverse, it's weird
        intervals = intervals.reindex(index=intervals.index[::-1])
    img = intervals[['bar1','bar2','bar3']].plot(kind='barh',stacked=True, color=['w','g','w'], xticks = np.linspace(0,1,21), legend=False)
    return img

def makeMatchupsFromOverallBeatProbs(allRanks, overall_beat_probs):
    overall_beat_probs.name = "overall_beat_opponent_prob"
    overall_beat_probs_df = pandas.DataFrame(overall_beat_probs)
    mergedRanks = allRanks.merge(overall_beat_probs_df, left_on='PLAYER_CHAR_COPY', right_index=True)
    specific_opponent_probs = 3*mergedRanks['beat_opponent_prob'] - 2*mergedRanks['overall_beat_opponent_prob']
    matchups = specific_opponent_probs.unstack()
#    averagedMatchups = (matchups + (1 - matchups.T))/2.0
#    return averagedMatchups
#can't average, we're going for asymmetry
    return matchups

def main():
    matchupPayoffs = 2*matchups - 1
    allWinRates = getAllWinRates(matchupPayoffs,10)
    #Plot will output to postscript file
    img = plotIntervals(allWinRates,True,-0.02)
    img.get_figure().savefig('sf5_balancing.pdf')
    print(img)
    
if __name__ == "__main__":
    main()

Axes(0.125,0.1;0.775x0.8)


/Users/jpmallette/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:79: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [130]:
# installing packages
!pip install pulp

    100% |████████████████████████████████| 13.5MB 53kB/s 
    100% |████████████████████████████████| 972kB 669kB/s 
  Running setup.py bdist_wheel for pulp ... - \ done
  Stored in directory: /Users/jpmallette/Library/Caches/pip/wheels/97/df/a8/eec45a8540c51ee9a1d153ef1eb83dc682bd22b25e09b4c334
  Running setup.py bdist_wheel for pyparsing ... - done
  Stored in directory: /Users/jpmallette/Library/Caches/pip/wheels/a5/6a/b2/2e2e0422eb4b72ead13570e09a22d6dfdf8219225b4b71bb3e
Successfully built pulp pyparsing
  Found existing installation: pyparsing 2.0.3
    DEPRECATION: Uninstalling a distutils installed project (pyparsing) has been deprecated and will be removed in a future version. This is due to the fact that uninstalling a distutils project will only partially uninstall the project.
    Uninstalling pyparsing-2.0.3:
      Successfully uninstalled pyparsing-2.0.3
